# Microphysics Simulation
Now that you learned how to run a simulation with fuse from a geant4 file up to `raw_records` we will take a look the the data that fuse can produce in the intermediate simulation steps focusing on the microphysics simulation. This part of the XENONnT simulation chain was formerly done using the `epix` software.

## Imports & Simulation Context
Just like in the Getting_Started notebook we will first do our necessary imports and then set up the full chain simulation context.  

In [ ]:
import fuse

In [ ]:
st = fuse.context.xenonnt_fuse_full_chain_simulation(
    output_folder="./fuse_data",
    simulation_config="sr0_dev",
)

st.set_config(
    {
        "path": "/project2/lgrandi/xenonnt/simulations/testing",
        "file_name": "pmt_neutrons_100.root",
        "entry_stop": 10,
    }
)

run_number = "00000"

## Running the Simulation
Just like in the Getting_Started notebook we will run the simulation using the `st.make(run_number, "target")` function from strax. This time we will tell fuse explicitly to save the intermediate simulation output. 

### Reading the root file and assigning the cluster index
Before we do some simulation and start to calculate stuff, we need to read in the root file from Geant4 and convert it into a data format that can be handled by strax. This step is done in the `ChunkInput` plugin and the result can be accessed using target `geant4_interactions`. This plugin used uproot to open the root file and then converts it into numpy arrays. Each event is then assigned a time based on the config option `source_rate`. The plugin will cut delayed interaction and divide the data into chunks if necessary. 

In the next simulation step we will give each interaction a cluster index. This is done by the `FindCluster` plugin. First, all interactions are grouped by time and then a DBSCAN algorithm is used to spacially cluster the interactions. The output of this plugin can be accessed using target `cluster_index` and can be loaded along with the `geant4_interactions`.

In [ ]:
st.make(run_number, "geant4_interactions")
st.make(run_number, "cluster_index")

geant4_interactions = st.get_df(run_number, ["geant4_interactions", "cluster_index"])

Lets take a look at the data:

In [ ]:
geant4_interactions.head(10)

## Clustering, Physics, and Cuts

### Clustering

In the first step, all interactions with the same `cluster_id` are merged.  
The cluster energy is computed as the sum of the individual interaction energies, while the cluster position and time are calculated as energy-weighted averages of the contributing interactions.

The interaction type of a cluster is derived from the constituent interactions (for example from the interaction with the highest energy or from the first interaction in the cluster). This interaction type is later used to select the appropriate emission model.


In [ ]:
st.make(run_number, "clustered_interactions")

### Volume Properties

After clustering, the VolumeProperties plugin assigns physical properties to each interaction based on its location in the detector. These properties include, for example, the local xenon density and whether charge extraction and S2 production are possible.

In addition, a vol_id is assigned to each interaction, identifying the detector volume it belongs to. The volume definitions are stored in a dictionary in fuse.common.

In [ ]:
st.make(run_number, "volume_properties")

### Electric Field and Yields

The next step is to model scintillation and ionization at the interaction site.

First, the local electric field strength is estimated using a simulated field map in the ElectricField plugin. The resulting field values can be accessed via the electric_field_values target.

Next, an emission model is used to estimate the number of produced photons and electrons. By default, fuse uses the NestYields plugin, which relies on nestpy. Alternative emission models (e.g. BBF or beta-response models) are also available, but should only be used with care.

The output of the emission model is stored in the quanta target.

In [ ]:
st.make(run_number, "electric_field_values")
st.make(run_number, "quanta")

### Cuts and Selections

After all physical quantities have been computed, selection criteria are evaluated.
The most basic selection is the VolumeSelection, which flags interactions located in detector regions of interest (by default the TPC and the region below the cathode).

Additional cuts can optionally be applied, such as an EnergyCut or an NRCut, depending on the simulation configuration. These cut plugins do not modify the data themselves, but instead produce boolean masks indicating which interactions pass each selection.

### MicroPhysicsSummary

Finally, the MicroPhysicsSummary plugin collects the results of the previous steps. This plugin merges all relevant inputs (clustered interactions, volume properties, electric field values, quanta, and cut results) into a single data set.

At the same time, it applies the configured selections and cuts, filtering out interactions that do not pass all required criteria. The resulting filtered and merged output can be accessed via the microphysics_summary target.

In [ ]:
st.make(run_number, "microphysics_summary")
microphysics_summary = st.get_df(run_number, "microphysics_summary")